# ASMSA: Rum molecular dynamics with CV derived AAE model

**Previous steps**
- [prepare.ipynb](prepare.ipynb): Download and sanity check input files
- [train.ipynb](train.ipynb): Use results of previous tuning in more thorough training|
- [tune.ipynb](tune.ipynb): Perform initial hyperparameter tuning for this molecule



In [ ]:
# local runs: avoid TF to consume GPU memory
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
tf.config.list_logical_devices()

In [ ]:
import asmsa
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import gromacs as gmx

In [ ]:
ompthreads=2
mpiranks=1

In [ ]:
# Kubernetes deployment
mdrunner=gmx.MDrunnerK8s()

def mdrun(**kwargs):
    mdrunner.run(pre={'cores':ompthreads*mpiranks,'gpus':1}, mdrunargs={**kwargs,'ntomp':ompthreads,'pin':'on'},ncores=mpiranks)

In [ ]:
# alternative local deployment
mdrunner=gmx.run.MDrunner()

# XXX: no MPI support so far
def mdrun(**kwargs):
    mdrunner.run(mdrunargs={**kwargs,'ntomp':ompthreads,'pin':'on'})

In [ ]:
# unlike previous notebooks, we use the initial PDB only

pdb = "trpcage_correct.pdb"
base = "trpcage"
topf=base+'.top'
grof=base+'.grp'

In [ ]:
gmx.pdb2gmx(f=pdb,o=grof,p=topf,water='tip3p',ff='amber94')

In [ ]:
# Dodecahedron box of suitable size (adjust eventually)

mdbox=2.0
gmx.editconf(f=grof,o=f"{base}-box.gro",c=True,d=str(mdbox),bt="dodecahedron")

In [ ]:
# Solvate
gmx.solvate(cp=f"{base}-box.gro",cs="spc216.gro",o=f"{base}-solv.gro",p=f"{base}.top")

In [ ]:
# Add ions
gmx.grompp(f='ions.mdp',c=f"{base}-solv.gro",p=topf,o="ions.tpr")

In [ ]:
gmx.select(s="ions.tpr",on='sol.ndx',select='SOL')
gmx.genion(s="ions.tpr",n="sol.ndx",o=f"{base}-ions.gro",p=topf,pname="NA",nname="CL",neutral=True)

In [ ]:
# Energy minimization

#gmx.start(f"grompp -f minim-sol.mdp -c {base}-ions.gro -p {base}.top -o em.tpr",wait=True,delete=True)
gmx.grompp(f="minim-sol.mdp",c=f"{base}-ions.gro",p=topf,o="em.tpr")

In [ ]:
# This job may run a bit longer, start it in background, with more cores and GPU

#gmx.start(f"mdrun -v -deffnm em -pin on",cores=4,gpus=1)
mdrun(deffnm="em")

In [ ]:
# Isothermal-isochoric equilibration, following the same job pattern
gmx.grompp(f="nvt.mdp",c="em.gro",r="em.gro",p=topf,o="nvt.tpr")

In [ ]:
mdrun(deffnm='nvt')

In [ ]:
# And isothermal-isobaric equilibration
gmx.grompp(f='npt.mdp',c='nvt.gro',r='nvt.gro',t='nvt.cpt',p=topf,o='npt.tpr')

In [ ]:
mdrun(deffnm='npt')

In [ ]:
# Extract temperature, pressure and density from the trajectories and plot them
gmx.energy(f='npt.edr',o='press.xvg',input='Pressure')
gmx.energy(f='npt.edr',o='dens.xvg',input='Density')
gmx.energy(f='npt.edr',o='temp.xvg',input='Temperature')

In [ ]:
temp = np.loadtxt('temp.xvg',comments=['#','@'])
press = np.loadtxt('press.xvg',comments=['#','@'])
dens = np.loadtxt('dens.xvg',comments=['#','@'])

plt.figure(figsize=(15,9))
plt.subplot(311)
plt.plot(press[:,0],press[:,1])
plt.title('isothermal-isobaric equilibration')
plt.grid()
#plt.xlabel('time (ps)')
plt.ylabel("pressure (bar)")

plt.subplot(312)
plt.ylabel('density (kg/m3)')
plt.grid()
plt.plot(dens[:,0],dens[:,1])

plt.subplot(313)
plt.xlabel('time (ps)')
plt.ylabel('temperature (K)')
plt.grid()
plt.plot(temp[:,0],temp[:,1])

plt.show()

In [ ]:
# Now we are ready to run metadynamics with plumed.dat generated in train.ipynb
# Checkpoints are stored, hence the simulation can be restarted if it gets killed

!cp md.mdp.template md.mdp

# 200 ns
# mdsteps=100000000

# 10 ns
mdsteps=5000000

# 100 ps
#mdsteps=50000

with open('md.mdp','a') as mdp:
    mdp.write("nsteps = %d\n" % mdsteps)

gmx.grompp(f='md.mdp',c='npt.gro',t='npt.cpt',p=topf,o='md.tpr')

In [ ]:
!touch HILLS
mdrun(deffnm='md',plumed='plumed.dat')
#mdrun(deffnm='md')